In [1]:
import random
import uuid
from typed_lists import TypedList, _initialize_global_thread_pool

In [2]:
# can set number of threads to use for parallel operations
# _initialize_global_thread_pool(1)

In [3]:
# examples of creating a typed list 
print(TypedList(range(10)) * TypedList(range(10)))
print(TypedList(["a", "b", "c", "d", "e"]) + TypedList(["f", "g", "h", "i", "j"]))
print(TypedList([1.0, 2.0, 3.0, 4.0, 5.0]) / TypedList([1.0, 2.0, 3.0, 4.0, 5.0]))

IntTypedList([0, 1, 4, 9, 16]...)
StringTypedList (["af", "bg", "ch", "di", "ej"])
FloatTypedList ([1.0, 1.0, 1.0, 1.0, 1.0])


In [4]:
# parallelized sorting
uuids = TypedList([str(uuid.uuid4()) for _ in range(1_000_000)])
uuids.sort()
uuids

StringTypedList(["00000c6a-a10c-41ca-8b99-1da2e4bfc676", "00001f05-4d64-478d-a7d1-b20e62ee2f49", "00001fbd-4ff8-40e2-8baf-cead79898413", "000023b5-753f-4295-8041-19abc5a3fc5b", "0000260b-3b6a-42fe-a893-ef9f3affb5bd"]...)

In [5]:
# parallelized counting
ids = TypedList([random.randint(0, 4) for _ in range(1_000_000)])
ids.count()

{1: 199784, 2: 200487, 3: 199877, 0: 199576, 4: 200276}

In [6]:
# broadcasting and parallelized operations
a = TypedList(range(1_000_000))
b = TypedList(range(1_000_000))
a + b * 3

IntTypedList([0, 4, 8, 12, 16]...)